<hr style="height:3px;border:none;color:#333;background-color:#333;" />
<img style=" float:right; display:inline" src="http://opencloud.utsa.edu/wp-content/themes/utsa-oci/images/logo.png"/>

### **University of Texas at San Antonio** 
<br/>
<br/>
<span style="color:#000; font-family: 'Bebas Neue'; font-size: 2.5em;"> **Open Cloud Institute** </span>

<hr style="height:3px;border:none;color:#333;background-color:#333;" />

### Lesson 3: MPI


### Swanand Mhalagi, Research Assistant 
#### swanand.mhalagi@utsa.edu 

**Open Cloud Institute, University of Texas at San Antonio, San Antonio, Texas, USA**
<hr style="height:3px;border:none;color:#333;background-color:#333;" />

### Trapezoidal Rule:

Now that we understand basic communication in MPI, we will proceed by parallelizing our first algorithm–numerical integration using the “trapezoid rule.” Early on in most calculus classes, students learn to estimate integrals using the trapezoid rule. A range to be integrated is divided into many vertical slivers, and each sliver is approximated with a trapezoid. The area of each trapezoid is computed, and then all their areas are added together.

In [5]:
%%bash
cat <<'EOF'> trapSerial.py
#In Python, a simple serial formulation of the trapezoidal rule would be as follows:
#trapSerial.py
#example to run: python trapSerial.py 0.0 1.0 10000

import numpy
import sys

#takes in command-line arguments [a,b,n]
a = float(sys.argv[1])
b = float(sys.argv[2])
n = int(sys.argv[3])

def f(x):
        return x*x

def integrateRange(a, b, n):
        '''Numerically integrate with the trapezoid rule on the interval from
        a to b with n trapezoids.
        '''
        integral = -(f(a) + f(b))/2.0
        # n+1 endpoints, but n trapazoids
        for x in numpy.linspace(a,b,n+1):
                integral = integral + f(x)
        integral = integral* (b-a)/n
        return integral

integral = integrateRange(a, b, n)
print "With n =", n, "trapezoids, our estimate of the integral from", a, "to", b, "is", integral
EOF

To run the above program and calculate the integral for the limits 0.0 to 1.0 by splitting the trapezoid in to 10000 pices, we will use the command mentioned below

In [6]:
!python trapSerial.py 0.0 1.0 10000

With n = 10000 trapezoids, our estimate of the integral from 0.0 to 1.0 is 0.333333335


### Parallelizing the Trapezoidal Rule:

The most important step in parallelizing an algorithm is finding the independent computations. This is the first step of architecting a parallel algorithm. With the trapezoidal rule, it’s easy to see that the calculation of the area of trapezoid can be performed independently of any other trapezoid, and so dividing the data at the trapezoid level seems natural. The code divides up the interval into the calculation of n trapezoids. To prallelize this process, we will divide the interval into n trapeziods and then divide up those n trapezoids to be calculated among the number of processors, size. Look at the following code: 

In [7]:
%%bash
cat <<'EOF'> trapParallel.py
#trapParallel.py
#example to run: mpiexec -n 4 python trapParallel.py 0.0 1.0 10000
import numpy
import sys
from mpi4py import MPI
from mpi4py.MPI import ANY_SOURCE

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

#takes in command-line arguments [a,b,n]
a = float(sys.argv[1])
b = float(sys.argv[2])
n = int(sys.argv[3])

#we arbitrarily define a function to integrate
def f(x):
        return x*x

#this is the serial version of the trapezoidal rule
#parallelization occurs by dividing the range among processes
def integrateRange(a, b, n):
        integral = -(f(a) + f(b))/2.0
        # n+1 endpoints, but n trapazoids
        for x in numpy.linspace(a,b,n+1):
                        integral = integral + f(x)
        integral = integral* (b-a)/n
        return integral


#h is the step size. n is the total number of trapezoids
h = (b-a)/n
#local_n is the number of trapezoids each process will calculate
#note that size must divide n
local_n = n/size

#we calculate the interval that each process handles
#local_a is the starting point and local_b is the endpoint
local_a = a + rank*local_n*h
local_b = local_a + local_n*h

#initializing variables. mpi4py requires that we pass numpy objects.
integral = numpy.zeros(1)
recv_buffer = numpy.zeros(1)

# perform local computation. Each process integrates its own interval
integral[0] = integrateRange(local_a, local_b, local_n)

# communication
# root node receives results from all processes and sums them
if rank == 0:
        total = integral[0]
        for i in range(1, size):
                comm.Recv(recv_buffer, ANY_SOURCE)
                total += recv_buffer[0]
else:
        # all other process send their result
        comm.Send(integral, dest = 0)

# root process prints results
if comm.rank == 0:
        print "With n =", n, "trapezoids, our estimate of the integral from", a, "to", b, "is", total
EOF

The parallel approach to trapezoidal integral estimation starts by dividing the original range among the processors. Each process will get a group of one or more trapezoids to calculate area over. Now, notice how we decided to implement to division of trapezoids among the processes. The processors individually calculate their own ranges to work on. Although this is a small detail, it is fairly important. We could have written the algorithm such that process 0 divides up the work for the other processors, then each processor calculates its area, and finally a sum is computed. However, this would introduce an unnecessary bottleneck: all processes with rank greater than 0 would be waiting for its data range to arrive. By having each process calculate its own range, we gain a large speedup. To run the above program and calculate the integral for the limits 0.0 to 1.0 by splitting the trapezoid in to 10000 pices, we will use the command mentioned below

In [8]:
!mpiexec -n 4 -mca btl ^openib python trapParallel.py 0.0 1.0 10000

With n = 10000 trapezoids, our estimate of the integral from 0.0 to 1.0 is 0.333333335
